In [1]:
!pip install selenium
!pip install beautifulsoup4

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time

# Initialize the browser
driver = webdriver.Chrome()

# Go to the website
driver.get("https://www.homes.com/")  # Replace with the actual URL

# Wait for the body tag to load to ensure the page is ready
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.TAG_NAME, "body"))
)

# Locate the search box using the class name
search_box = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.CLASS_NAME, "multiselect-search"))  # Adjust if needed
)

# Ensure the input is in view and interactable
driver.execute_script("arguments[0].scrollIntoView();", search_box)

# Send search query
search_box.send_keys(" Douglas County")
search_box.send_keys(Keys.RETURN)

# Wait for the initial results container to load
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "placardContainer")) 
)

# Now capture the page source after the search has resulted in new content
page_source = driver.page_source

# Parse the page with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

print(soup.title.text)

# Close the browser
# driver.quit()

Douglas County CO Real Estate & Homes for Sale - Homes.com


In [3]:
# Extract the information from the placard-container
property_list = []

# Loop through each property placard on the page
placards = soup.find_all('li', class_='placard-container')
for placard in placards:
    # Initialize a dictionary to store the property details
    property_details = {}

    # Get the price
    price = placard.find('p', class_='price-container').text.strip()
    property_details['price'] = price

    # Extract property features
    details = placard.find('ul', class_='detailed-info-container')
    if details:
        features = details.find_all('li')
        for feature in features:
            text = feature.text.strip()
            if "Beds" in text:
                property_details['beds'] = text
            elif "Baths" in text:
                property_details['baths'] = text
            elif "Sq Ft" in text:
                property_details['sq_ft'] = text

    

    # Extract the property address
    address = placard.find('p', class_='property-name').text.strip()
    property_details['address'] = address

    # Extract the description of the property
    description = placard.find('p', class_='property-description').text.strip()
    property_details['description'] = description

    agent = placard.find('div', class_='agent-info-container')
    if agent:
        # Extract agent information
        agent_name = placard.find('p', class_='agent-name').text.strip()
        agency_name = placard.find('p', class_='agency-name').text.strip()
        agent_number = placard.find('p', class_='agency-number').text.strip()

        property_details['Agent Name'] = agent_name
        property_details['Agency Number'] = agency_name
        property_details['Agent Number'] = agent_number

        # Store the property details in the list
        property_list.append(property_details)

# Output the scraped information
for property in property_list:
    for key, value in property.items():
        print(f"{key}: {value}")
    print()

driver.quit()

price: $1,295,000
beds: 3 Beds
baths: 2.5 Baths
sq_ft: 2,546 Sq Ft
address: 1641 Castlewood Dr, Franktown, CO 80116
description: Options abound with this property.  The well permit and County Zoning allow for an additional home to be built on this land.  The home size is not restricted.  You can convert the barn or arena into a house or construct a separate residence or ADU.  A large 70’ x 120’ building can be used as an equestrian facility or storage for RVs, boats, autos, etc. Or it can be rented out.  The property has
Agent Name: Cindy Rein
Agency Number: Sherlock Homes
Agent Number: (855) 941-4250

price: $1,125,000
beds: 5 Beds
baths: 5.5 Baths
sq_ft: 5,371 Sq Ft
address: 2905 Morningbird Ln, Castle Rock, CO 80109
description: Discover the perfect blend of luxury, comfort, and functionality in this stunning 5 bedroom, 5.5 bath, ranch-style home located in The Meadows of Castle Rock. With over 5,800 square feet, this expansive home is designed to accommodate all your needs, whether